In [14]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import numpy as np
import healpy as hp
from astropy.io import fits
from pprint import pprint
from tqdm import tqdm
import pickle
import corner
import os

from scipy import optimize
from scipy.stats import poisson

import jax
import jax.numpy as jnp

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../utils/matplotlibrc')

from utils import ed_fcts_amarel as ef
from utils import create_mask as cm
from utils import ed_plotting as eplt

from numpyro.infer.initialization import init_to_sample #new

In [15]:
# load GPU
gpu_id = '3'
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id

In [16]:
# name of the synthetic directory
sim_name = 'canon_g1p2_ola_v2'

# load SVI fit results (these should be the only parameters that you are loading)
sim_id = 7.1234567
temp_id = 5.23457
gp_id = 1.16
blg_id = -1
mod_id = 11
svi_id_1 = 671 # (671,672) =  (Fit 1, Fit 2)
sim_seed = 1000
svi_seed = 0

# load data directory
data_dir = ef.load_data_dir(sim_name)
os.system("mkdir -p "+data_dir)

0

In [17]:
# import all saved model parameters from python file
fit_filename, module_name = ef.generate_fit_filename_from_ids(sim_id, temp_id, gp_id, blg_id, mod_id, svi_id_1, sim_seed, svi_seed)
fit_dir = data_dir + 'fits/' + fit_filename + '/'

sys.path.append(fit_dir)
import importlib 
module = importlib.import_module(module_name)

# Load all the variables from the module
globals().update(vars(module))

# load saved data from GP -> Data fit
fit_file_name = ('ebin' + str_ebin + '_smp_svi_' + 
            str_lr + '_' + str_n_steps + '_' + 
                str_guide + '_' + str_num_particles + '_' + 
                str_sim_seed + '_' + str_svi_seed + '.p')

samples_dict_1, svi_results_1 = pickle.load(open(fit_dir + fit_file_name, 'rb'))

# generate temp_sample_dict
all_temp_names = ['iso', 'psc', 'bub', 'pib', 'ics', 'blg', 'nfw', 'dsk', 'gp']
names = list(samples_dict_1.keys())
temp_sample_dict_1 = {k: samples_dict_1[k] for k in all_temp_names if k in names}

In [18]:
# name of the synthetic directory
sim_name = 'canon_g1p2_ola_v2'

# load SVI fit results (these should be the only parameters that you are loading)
sim_id = 7.1234567
temp_id = 5.23457
gp_id = 1.16
blg_id = -1
mod_id = 11
svi_id_2 = 672 # (671,672) =  (Fit 1, Fit 2)
sim_seed = 1000
svi_seed = 0

# load data directory
data_dir = ef.load_data_dir(sim_name)
os.system("mkdir -p "+data_dir)

0

In [20]:
svi_id_2 = 672 # (671,672) =  (Fit 1, Fit 2)
fit_filename, module_name = ef.generate_fit_filename_from_ids(sim_id, temp_id, gp_id, blg_id, mod_id, svi_id_2, sim_seed, svi_seed)
fit_dir = data_dir + 'fits/' + fit_filename + '/'

sys.path.append(fit_dir)
import importlib 
module = importlib.import_module(module_name)

# Load all the variables from the module
globals().update(vars(module))

# load saved data from GP -> Data fit
fit_file_name = ('ebin' + str_ebin + '_smp_svi_' + 
            str_lr + '_' + str_n_steps + '_' + 
                str_guide + '_' + str_num_particles + '_' + 
                str_sim_seed + '_' + str_svi_seed + '.p')

samples_dict_2, svi_results_2 = pickle.load(open(fit_dir + fit_file_name, 'rb'))
temp_sample_dict_2 = {k: samples_dict_2[k] for k in all_temp_names if k in names}

# load loss data
loss_file_name = ('ebin' + str_ebin + '_loss_data_' + 
            str_lr + '_' + str_n_steps + '_' + 
                str_guide + '_' + str_num_particles + '_' + 
                str_sim_seed + '_' + str_svi_seed + '.p')
losses_1, ll_list_1, losses_2, ll_list_2 = pickle.load(open(fit_dir + loss_file_name, 'rb'))

[autoreload of __main__ failed: Traceback (most recent call last):
  File "/home/edr76/.conda/envs/jax/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/edr76/.conda/envs/jax/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/edr76/.conda/envs/jax/lib/python3.11/importlib/__init__.py", line 148, in reload
    raise ImportError(msg.format(name), name=name)
ImportError: module settings_7p1234567_5p23457_1p16_-1_11_672_1000_0 not in sys.modules
]
[autoreload of __main__ failed: Traceback (most recent call last):
  File "/home/edr76/.conda/envs/jax/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/edr76/.conda/envs/jax/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 471, in superrel

In [21]:
# additional loss information for plotting
Nstep1 = len(losses_1)
Nstep2 = len(losses_2)

rx1 = ( np.array(svi_results_1.recorded_steps)  ) / Nstep1
rx2 = ( np.array(svi_results_2.recorded_steps) ) / Nstep2

In [22]:
# load true data if exits
if data_file != 'fermi_data_sum':
    temp_dict = np.load(data_dir + 'all_templates_ebin' + str(ebin)  + '.npy', allow_pickle=True).item()
else:
    pass

In [23]:
pickle.dump(
    (temp_dict, temp_sample_dict_1, temp_sample_dict_2, losses_1, ll_list_1, losses_2, ll_list_2, rx1, rx2),
    open('../figures/data/appdata_degeneracy.p', 'wb')
)

: 